In [ ]:
import torch
import torch.nn as nn
import os
import matplotlib.image as mtimage
import matplotlib.pyplot as plt
import numpy as np
import random
import json
import re, os, sys, json, random
from tqdm import tqdm
from PIL import Image
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset,DataLoader
import torchvision
from torchvision import transforms
import pandas as pd
import shutil
import torchvision.datasets as dset

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
torch.cuda.empty_cache()

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
root_train= '/content/drive/MyDrive/capstonedataset/COVID/99-1/train'
root_valid= '/content/drive/MyDrive/capstonedataset/COVID/99-1/test'

In [ ]:
train_dataset = dset.ImageFolder(root_train,

                          transforms.Compose([
   #transforms.ToPILImage(),
   transforms.Resize(256),
   transforms.CenterCrop(224),
   #transforms.RandomHorizontalFlip(),
   transforms.ToTensor(),
   #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]))

train_data=DataLoader(train_dataset,batch_size=128,shuffle=True,num_workers=12)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:

test_dataset=dset.ImageFolder(root_valid,
                              transforms.Compose([
   #transforms.ToPILImage(),
   transforms.Resize(256),
   transforms.CenterCrop(224),
   #transforms.RandomHorizontalFlip(),
   transforms.ToTensor(),
   #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]))
val_data=DataLoader(test_dataset,batch_size=128,num_workers=12)

In [ ]:
torch.cuda.empty_cache()
def train(model,train_data,val_data,save_name,epoch,early_stop=None,frozen_grad=False):

    model.to(device)
    #optim=torch.optim.SGD(model.parameters(),lr=0.0005,momentum=0.9,weight_decay=0.0001)


    optim=torch.optim.Adam(model.parameters(),
                    lr=0.0005,
                    betas=(0.9, 0.999),
                    eps=1e-08,
                    weight_decay=0.0001,
                  amsgrad=False)

    loss_function=torch.nn.CrossEntropyLoss()
    epoch=epoch
    best_acc=0
    record=[]

    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau( optim,
    mode="min",
    factor=0.5,
    patience=1,
    threshold=0.0001,
    threshold_mode="rel",
    cooldown=0,
    min_lr=0,
    eps=1e-08,
    verbose=False,
)
    print('start training ...')

    for e in range(epoch):
        #train
        model.train()

        for i ,j in train_data:
            i=i.to(torch.float).to(device)
            j=j.to(device)
            output=model(i)
            optim.zero_grad()
            loss=loss_function(output,j)
            loss.backward()
            optim.step()

        if frozen_grad==True:

               if e == 5:
                    #finetune_param = False
                    for name, param in model.named_parameters():
                        param.requires_grad_(True)
        scheduler.step(loss)
        # valid
        model.eval()
        with torch.no_grad():
            c=0
            t=0
            for img,target in val_data:
                img=img.to(torch.float).to(device)
                target=target.to(device)
                pred=model(img)
                result=torch.argmax(pred,dim=1)
                for index in range(len(result)):
                    t+=1
                    if result[index]==target[index]:
                        c+=1
            val_acc=c/t
            val_loss=loss_function(pred,target)
            print('epoch:{}'.format(e),'\n',"ACC {:.3%},train_loss {:.3f}".format(val_acc,loss))
        #early stop
        if early_stop:
            if val_acc > best_acc:
                best_acc = val_acc
                es = 0
                torch.save(model.state_dict(), "model_" + str(es) + 'weight.pt')
            else:
                es += 1
                print("Counter {} of 7".format(es))

                if es > 7:
                    print("Early stopping with best_acc:{:.2%}  and val_acc for this epoch:{:.3f} ".format(best_acc,val_acc))
                    break
        record.append(val_acc)
    torch.save(record,save_name)

In [ ]:
#model1=torchvision.models.resnet50(weights=torchvision.models.ResNet50_Weights.IMAGENET1K_V1)
model_1=torchvision.models.resnet50(weights=False)
model_1.fc=nn.Linear(in_features=2048, out_features=2, bias=True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [ ]:
train(model_1,train_data,val_data,'resnet50_CT_1%_record.pth',3,frozen_grad=False)

start training ...
epoch:0 
 ACC 68.643%,train_loss 0.539
epoch:1 
 ACC 68.854%,train_loss 0.543
epoch:2 
 ACC 68.685%,train_loss 0.719


In [ ]:
from collections import OrderedDict


class Res(nn.Module):
    def __init__(self, backbone, cls_num=3):
        super().__init__()
        hidden_dim=2048
        output_dim=2048
        self.backbone = backbone
        self.fc = torch.nn.Sequential(OrderedDict([
            ('fc1', torch.nn.Linear(hidden_dim, hidden_dim)),
            ('bn1', torch.nn.BatchNorm1d(hidden_dim)),
            ('relu1', torch.nn.ReLU(inplace=True)),
            ('fc2', torch.nn.Linear(hidden_dim, hidden_dim)),
            ('bn2', torch.nn.BatchNorm1d(hidden_dim)),
            ('relu2', torch.nn.ReLU(inplace=True)),
            ('fc3', torch.nn.Linear(hidden_dim, output_dim)),
            #('bn3', torch.nn.BatchNorm1d(output_dim)),
            #('relu3', torch.nn.ReLU(inplace=True)),
            ('fc4', torch.nn.Linear(output_dim, cls_num))
        ]))

    def forward(self, x):
        x0 = self.backbone(x).flatten(start_dim=1)
        out = self.fc(x0)
        return out

# model_tes=torchvision.models.resnet50(weights=False)
#resnet = torchvision.models.resnet50(weights=False)
# resnet = torchvision.models.resnet18()
backbone = nn.Sequential(*list(resnet.children())[:-1])
model = Res(backbone, 3)#记得改标签数量

state_dic=torch.load('/content/drive/MyDrive/parameter_new/byol_covid10.pth')
# print(state_dic.keys)
# print(model_tes)
model.backbone.load_state_dict(state_dic) #加载预训练参数到backbone
# pred = model(x)


<All keys matched successfully>

In [ ]:
print(state_dic)

OrderedDict([('0.weight', tensor([[[[ 7.7497e-03, -1.0883e-02, -3.3928e-02,  ...,  3.5883e-02,
            2.1494e-02,  2.0480e-03],
          [-2.6684e-03, -2.1879e-03, -1.8651e-02,  ..., -2.4305e-02,
           -2.1250e-02, -1.9293e-02],
          [-5.7261e-02, -1.2464e-02,  4.2723e-03,  ..., -4.0637e-02,
           -5.1305e-03, -1.6824e-02],
          ...,
          [ 1.0040e-02, -4.7528e-02, -2.2871e-02,  ...,  2.5664e-02,
            1.7654e-02,  3.9635e-02],
          [ 9.4889e-03, -4.2904e-03,  2.1152e-02,  ...,  2.1197e-02,
            5.9081e-03,  1.0198e-03],
          [-9.1288e-03,  6.5377e-03,  6.0506e-03,  ..., -3.6315e-02,
            2.6605e-02, -3.7084e-03]],

         [[-9.7054e-03,  2.9771e-03, -2.2454e-02,  ..., -8.3752e-03,
           -3.8284e-02, -6.3246e-02],
          [-7.0762e-02, -6.0363e-02, -4.3605e-02,  ..., -3.9677e-03,
           -1.5632e-02, -2.6792e-02],
          [-3.7319e-02,  2.0025e-03, -4.4641e-02,  ..., -4.7878e-02,
            3.5613e-03, -6.1997e

In [ ]:
train(model,train_data,val_data,'/content/drive/MyDrive/parameter_new/results_new/BYOL_covid1',100,frozen_grad=False)

start training ...
epoch:0 
 ACC 44.393%,train_loss 1.112
epoch:1 
 ACC 44.393%,train_loss 6.164
epoch:2 
 ACC 40.306%,train_loss 2.908
epoch:3 
 ACC 43.271%,train_loss 1.259
epoch:4 
 ACC 33.172%,train_loss 1.090
epoch:5 
 ACC 39.296%,train_loss 1.202
epoch:6 
 ACC 33.007%,train_loss 3.582
epoch:7 
 ACC 28.229%,train_loss 0.860
epoch:8 
 ACC 30.290%,train_loss 0.798
epoch:9 
 ACC 31.772%,train_loss 0.766
epoch:10 
 ACC 32.841%,train_loss 0.667
epoch:11 
 ACC 32.215%,train_loss 0.724
epoch:12 
 ACC 32.133%,train_loss 0.561
epoch:13 
 ACC 35.835%,train_loss 0.743
epoch:14 
 ACC 40.973%,train_loss 0.539
epoch:15 
 ACC 43.040%,train_loss 1.820
epoch:16 
 ACC 46.548%,train_loss 0.454
epoch:17 
 ACC 54.096%,train_loss 0.349
epoch:18 
 ACC 53.481%,train_loss 0.363
epoch:19 
 ACC 53.245%,train_loss 0.221
epoch:20 
 ACC 54.828%,train_loss 0.244
epoch:21 
 ACC 55.448%,train_loss 0.095
epoch:22 
 ACC 55.607%,train_loss 0.092
epoch:23 
 ACC 62.098%,train_loss 0.437
epoch:24 
 ACC 63.503%,train_lo

KeyboardInterrupt: 

In [ ]:
import torchvision
from torchvision import transforms
resnet = torchvision.models.resnet50(weights=False)

In [ ]:
resnet.conv1.state_dict()

In [ ]:
print(model)

In [ ]:
print(resnet)

In [ ]:
import torch

# 假设 model 是你的模型实例，且你已经定义了它
print("加载前的第一层权重:", next(resnet.parameters()).data)


加载前的第一层权重: tensor([[[[ 7.7497e-03, -1.0883e-02, -3.3928e-02,  ...,  3.5883e-02,
            2.1494e-02,  2.0480e-03],
          [-2.6684e-03, -2.1879e-03, -1.8651e-02,  ..., -2.4305e-02,
           -2.1250e-02, -1.9293e-02],
          [-5.7261e-02, -1.2464e-02,  4.2723e-03,  ..., -4.0637e-02,
           -5.1305e-03, -1.6824e-02],
          ...,
          [ 1.0040e-02, -4.7528e-02, -2.2871e-02,  ...,  2.5664e-02,
            1.7654e-02,  3.9635e-02],
          [ 9.4889e-03, -4.2904e-03,  2.1152e-02,  ...,  2.1197e-02,
            5.9081e-03,  1.0198e-03],
          [-9.1288e-03,  6.5377e-03,  6.0506e-03,  ..., -3.6315e-02,
            2.6605e-02, -3.7084e-03]],

         [[-9.7054e-03,  2.9771e-03, -2.2454e-02,  ..., -8.3752e-03,
           -3.8284e-02, -6.3246e-02],
          [-7.0762e-02, -6.0363e-02, -4.3605e-02,  ..., -3.9677e-03,
           -1.5632e-02, -2.6792e-02],
          [-3.7319e-02,  2.0025e-03, -4.4641e-02,  ..., -4.7878e-02,
            3.5613e-03, -6.1997e-02],
         

In [ ]:
print("加载后的第一层权重:", next(model.parameters()).data)

加载后的第一层权重: tensor([[[[ 7.7497e-03, -1.0883e-02, -3.3928e-02,  ...,  3.5883e-02,
            2.1494e-02,  2.0480e-03],
          [-2.6684e-03, -2.1879e-03, -1.8651e-02,  ..., -2.4305e-02,
           -2.1250e-02, -1.9293e-02],
          [-5.7261e-02, -1.2464e-02,  4.2723e-03,  ..., -4.0637e-02,
           -5.1305e-03, -1.6824e-02],
          ...,
          [ 1.0040e-02, -4.7528e-02, -2.2871e-02,  ...,  2.5664e-02,
            1.7654e-02,  3.9635e-02],
          [ 9.4889e-03, -4.2904e-03,  2.1152e-02,  ...,  2.1197e-02,
            5.9081e-03,  1.0198e-03],
          [-9.1288e-03,  6.5377e-03,  6.0506e-03,  ..., -3.6315e-02,
            2.6605e-02, -3.7084e-03]],

         [[-9.7054e-03,  2.9771e-03, -2.2454e-02,  ..., -8.3752e-03,
           -3.8284e-02, -6.3246e-02],
          [-7.0762e-02, -6.0363e-02, -4.3605e-02,  ..., -3.9677e-03,
           -1.5632e-02, -2.6792e-02],
          [-3.7319e-02,  2.0025e-03, -4.4641e-02,  ..., -4.7878e-02,
            3.5613e-03, -6.1997e-02],
         